In [1]:
import pickle
import math

In [2]:
with open("time_series.pkl", "rb") as f:
    data = pickle.load(f)

In [3]:
time_series = data["sub-292"]["ses-preop"]["run-01"]["time_series"]
TR = data["sub-292"]["ses-preop"]["run-01"]["TR"]

In [4]:
def sliceWindows(time_series, window_length, interval_second):
    frame = math.ceil(window_length/TR) # 滑动窗口选择时间通常在 30s~100s 之间
    interval = math.ceil(interval_second/TR)
    windows = []
    l, r = 0, frame
    while r < time_series.shape[0]:
        windows.append(time_series[l:r])
        l += interval
        r = l + frame
    return windows

In [5]:
window_length = 30 # 单位 s
interval_second = 2 # 步长，单位 s

windows = sliceWindows(time_series, window_length, interval_second)
print(len(windows), windows[0].shape)

165 (15, 116)


In [6]:
sliced_time_series = {}
for sub, ses in data.items():
    sliced_time_series[sub] = {}
    for op, runs in ses.items():
        sliced_time_series[sub][op] = {}
        for run, items in runs.items():
            windows = sliceWindows(items["time_series"], window_length, interval_second)
            if windows:
                sliced_time_series[sub][op][run] = {
                    "time_series" : items["time_series"],
                    "sliced_time_series" : windows
                }

In [7]:
with open("silced_windows.pkl", "wb") as f:
    pickle.dump(sliced_time_series, f)

In [ ]:
def load_data_and_save_siliding_window(load_path, save_path, window_length, interval_second):
    with open(load_path, "rb") as f:
        data = pickle.load(f)
    sliced_time_series = {}
    for sub, ses in data.items():
        sliced_time_series[sub] = {}
        for op, runs in ses.items():
            sliced_time_series[sub][op] = {}
            for run, items in runs.items():
                windows = sliceWindows(items["time_series"], window_length, interval_second)
                if windows:
                    sliced_time_series[sub][op][run] = {
                        "time_series" : items["time_series"],
                        "sliced_time_series" : windows
                    }
    with open(save_path, "wb") as f:
        pickle.dump(sliced_time_series, f)
    return sliced_time_series